# Imports

In [1]:
!pwd
import sys
from bumps.names import Curve, fit, FitProblem
from bumps.dream.state import load_state
import matplotlib.pyplot as plt
import numpy as np
import scipy.fft as fft
import os

sys.path.append("../../../../src/molgroups")
import molgroups as mol

sys.path.append("../../../../src/scatpy")
import rs
import rsdi

sys.path.append("../../../../src/infotheory")
import entropy

/Users/frank/Dropbox/My Mac (PN115993.campus.nist.gov)/Documents/programming/molgroups/examples/information_theory/entropy_gridsearch_singlethread/garefl


# Create Short Example Fit

In [9]:
!make clean
!make
!cp hk054.refl sim0.dat
!refl1d run.py --fit=dream --store=T --init=lhs --parallel --burn=100 --steps=5 --overwrite

rm -f setup.o
g++  -g -O2 -Wall  -fPIC -DUSE_DOUBLE -DReal=double -DHAVE_CONFIG_H -I/Users/frank/danse/refl1d/garefl/boxmin -I/Users/frank/danse/refl1d/garefl/model1d -I/Users/frank/danse/refl1d/garefl/src -c setup.cc -o setup.o
setup.cc:93:14: warning: unused variable 'freepars' [-Wunused-variable]
    fitpars *freepars = &fit[1].pars;
             ^
1 warning generated.
cd /Users/frank/danse/refl1d/garefl/boxmin && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
cd /Users/frank/danse/refl1d/garefl/model1d && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
cd /Users/frank/danse/refl1d/garefl/src && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
g++    -fPIC -o fit setup.o /Users/frank/danse/refl1d/garefl/src/ga_simul.o /Users/frank/danse/refl1d/garefl/src/ga.o -L/Users/frank/danse/refl1d/garefl/model1d -lrefl -L/Users/frank/danse/refl1d/garefl/b

# Defining Optimization Parameters in simpar.dat

Simpar.dat contains a list of all fit parameters with a designation, whether they are marginal (d) or nuisance (i) parameters. This is followed by the parameter name, the initial parameter value, and the fit boundaries. If three more numbers are given, this designates that an information content search over this parameter is performed (start, stop, step). A preceding f (fi or fd) at the beginning of the line indicates that the fit boundaries for such a search parameter are fixed (for example for volume fractions between 0 and 1), otherwise the fit boundary moves according to the varied parameter and the initally given fit boundaries.

In [3]:
%%writefile simpar.dat
i l_siox 18 10 30
i rho_siox 3.55e-6 3.20e-6 3.80e-6
i l_submembrane 2.5 1.0 10.0
d l_lipid1 11.8 10.0 15.0
d l_lipid2 12.4 10.0 15.0
d vf_bilayer 0.95 0.90 1.00
i rho_solv_0 6.34e-6 5.8e-6 6.4e-6 -0.5e-6 6.5e-6 0.5e-6
i global_rough 3.0 2.0 5.0
i sigma 2.5 2.0 5.0
i background_0 -8.0 -9.0 -5.0
n prefactor 2.58496

Overwriting simpar.dat


# Variables

In [4]:
# general fit setup
setupdir = os.getcwd()
runfile = "run"
store = 'T'
fitsource = "garefl"

# particular entropy setup
burn = 8000
steps = 500
convergence = 2.0
miniter = 2
mode = 'water'
bClusterMode = False
bFetchMode = False
time = 2
bcalcsymmetric = True
upper_info_plotlevel = None
plotlimits_filename = ""
calcsingle = False

# setup batchscript for SLURM (if used)
script = []
script.append('#!/bin/bash\n')
script.append('#SBATCH --job-name=entro {mcmc_iteration}\n')
script.append('#SBATCH -A mc4s9np\n')
script.append('#SBATCH -p RM\n')
script.append('#SBATCH -t 0' + str(time) + ':00:00\n')
script.append('#SBATCH -N 4\n')
script.append('#SBATCH --ntasks-per-node 28\n')
script.append('\n')
script.append('set +x\n')
script.append('cd $SLURM_SUBMIT_DIR\n')
# script.append('cd '+dirname+'\n')
script.append('\n')
script.append('module load python/2.7.11_gcc\n')
script.append('export PYTHONPATH=/home/hoogerhe/bin/lib/python2.7/site-packages:/home/hoogerhe/src/bumps\n')
script.append('\n')
script.append('mpirun -np 112 python /home/hoogerhe/src/refl1d/bin/refl1d_cli.py {mcmc_dirname}/run.py --fit=dream --mpi --init=lhs --batch --pop=28 --time=' 
              + str(float(time) - 0.1) + ' --thin=20 --store={mcmc_dirname}/save --burn=' + str(burn) 
              + ' --steps=' + str(steps) + '\n')


# Fit Setup

In [5]:
entr = entropy.Entropy(
    fitsource=fitsource,
    spath=setupdir,
    mcmcpath=store,
    runfile=runfile,
    mcmcburn=burn, 
    mcmcsteps=steps, 
    convergence=convergence, 
    miniter=miniter, 
    mode=mode,                      
    bClusterMode=bClusterMode, 
    bFetchMode=bFetchMode, 
    calc_symmetric=bcalcsymmetric,
    upper_info_plotlevel=upper_info_plotlevel, 
    plotlimits_filename=plotlimits_filename,
    slurmscript=script
)

g++  -g -O2 -Wall  -fPIC -DUSE_DOUBLE -DReal=double -DHAVE_CONFIG_H -I/Users/frank/danse/refl1d/garefl/boxmin -I/Users/frank/danse/refl1d/garefl/model1d -I/Users/frank/danse/refl1d/garefl/src -c setup.cc -o setup.o


setup.cc:93:14: warning: unused variable 'freepars' [-Wunused-variable]
    fitpars *freepars = &fit[1].pars;
             ^
1 warning generated.


cd /Users/frank/danse/refl1d/garefl/boxmin && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
cd /Users/frank/danse/refl1d/garefl/model1d && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
cd /Users/frank/danse/refl1d/garefl/src && /Applications/Xcode.app/Contents/Developer/usr/bin/make
make[1]: Nothing to be done for `all'.
g++    -fPIC -o fit setup.o /Users/frank/danse/refl1d/garefl/src/ga_simul.o /Users/frank/danse/refl1d/garefl/src/ga.o -L/Users/frank/danse/refl1d/garefl/model1d -lrefl -L/Users/frank/danse/refl1d/garefl/boxmin -lboxmin  -lm 
g++ -shared -o model.so setup.o /Users/frank/danse/refl1d/garefl/src/refl1d.o -L/Users/frank/danse/refl1d/garefl/model1d -lrefl  -lm 


In [6]:
entr.run_optimization()

In [7]:
entr.plot_results()